<a href="https://colab.research.google.com/github/areias/bert_covid_sentiment/blob/main/minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone -b master https://github.com/digitalepidemiologylab/covid-twitter-bert.git drive/MyDrive/covid-twitter-bert

Cloning into 'drive/MyDrive/covid-twitter-bert'...
remote: Enumerating objects: 1660, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1660 (delta 150), reused 140 (delta 88), pack-reused 1438
Receiving objects: 100% (1660/1660), 3.48 MiB | 10.01 MiB/s, done.
Resolving deltas: 100% (1050/1050), done.


In [28]:
!cp -r /content/drive/MyDrive/models-93490036e00f37ecbe6693b9ff4ae488bb8e9270/* drive/MyDrive/covid-twitter-bert/tensorflow_models/

## Install dependencies

In [3]:
!pip install -r drive/MyDrive/covid-twitter-bert/requirements.txt

     |████████████████████████████████| 516.2 MB 3.5 kB/s 
     |████████████████████████████████| 1.1 MB 53.5 MB/s 
     |████████████████████████████████| 170 kB 51.6 MB/s 
     |████████████████████████████████| 235 kB 52.4 MB/s 
     |████████████████████████████████| 1.2 MB 39.3 MB/s 
     |████████████████████████████████| 454 kB 54.9 MB/s 
     |████████████████████████████████| 2.9 MB 51.1 MB/s 
     |████████████████████████████████| 3.0 MB 29.5 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=52a440271cd989a8127b45be9624be118f25389406126ba677821659fdfcdc49
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estima

In [4]:
import tensorflow
print(tensorflow.__version__)

2.2.0


## Prepare dataset

In [5]:
import pandas as pd 
df=pd.read_csv("/content/drive/MyDrive/train-4.csv")

In [6]:
# id,label,text
df=df.loc[:,["tweet_id","label", "text"]]
df.columns=['id','label', 'text']

In [7]:
type(df['label'][0])

numpy.int64

In [8]:
df['label']=df['label'].apply(lambda x: str(x))

In [9]:
type(df['id'][0])

numpy.float64

In [10]:
df['id']=df['id'].apply(lambda x: int(x))

In [11]:
type(df['id'][0])


numpy.int64

In [12]:
df['label'][0]

'0'

In [13]:
import numpy as np
"""
    60% - train set,
    20% - dev/validation set,
    20% - test set"""

train, dev, test = np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [14]:
type(train['label'][0])

str

In [15]:
import os
os.makedirs("drive/MyDrive/covid-twitter-bert/data/finetune/originals/crowdbreaks")

In [16]:
train.to_csv("drive/MyDrive/covid-twitter-bert/data/finetune/originals/crowdbreaks/train.tsv", sep="\t",index=False)
dev.to_csv("drive/MyDrive/covid-twitter-bert/data/finetune/originals/crowdbreaks/dev.tsv", sep="\t",index=False)
test.to_csv("drive/MyDrive/covid-twitter-bert/data/finetune/originals/crowdbreaks/test.tsv", sep="\t",index=False)


In [17]:
!ls

drive  sample_data


In [18]:
!pip install spacy==3

     |████████████████████████████████| 12.7 MB 6.7 MB/s 
     |████████████████████████████████| 9.1 MB 15.3 MB/s 
     |████████████████████████████████| 628 kB 59.7 MB/s 
     |████████████████████████████████| 451 kB 50.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [19]:
import spacy 
print(spacy.__version__)

3.0.0


In [29]:
import sys
sys.path.append('drive/MyDrive/covid-twitter-bert')
sys.path.append('drive/MyDrive/covid-twitter-bert/tensorflow_models')


In [31]:
# had to change line 462 again in ctbert/tensorflow_models/official/nlp/data/classifier_data_lib.py
# also changed config.py line 32 to point to ct-bert model vs. 2
%%shell 
cd drive/MyDrive/covid-twitter-bert/preprocess 
python create_finetune_data.py --run_prefix test_run --finetune_datasets crowdbreaks --model_class bert_large_uncased_wwm --max_seq_length 96 --asciify_emojis --username_filler twitteruser --url_filler twitterurl --replace_multiple_usernames --replace_multiple_urls --remove_unicode_symbols

2021-11-24 14:56:07,013 [INFO ] [__main__    ]: Processing dataset crowdbreaks...
2021-11-24 14:56:07,016 [INFO ] [__main__    ]: Reading data for for type train...
2021-11-24 14:56:07,030 [INFO ] [__main__    ]: Creating preprocessed files...
2021-11-24 14:56:08,456 [INFO ] [__main__    ]: Reading data for for type dev...
2021-11-24 14:56:08,462 [INFO ] [__main__    ]: Creating preprocessed files...
2021-11-24 14:56:08,899 [INFO ] [__main__    ]: Creating tfrecords files...
2021-11-24 14:56:09,016 [INFO ] [absl        ]: Writing example 0 of 2041
2021-11-24 14:56:09,017 [INFO ] [absl        ]: *** Example ***
2021-11-24 14:56:09,017 [INFO ] [absl        ]: guid: train-0
2021-11-24 14:56:09,017 [INFO ] [absl        ]: tokens: [CLS] arizona classrooms vulnerable to me ##as ##les outbreak : the arizona department of health services looked at herd . . . twitter ##ur ##l [SEP]
2021-11-24 14:56:09,017 [INFO ] [absl        ]: input_ids: 101 5334 12463 8211 2000 2033 3022 4244 8293 1024 1996 

In [32]:
!rm -r /content/drive/MyDrive/covid-twitter-bert/data/finetune/run_2021-11-24_14-55-26_453313_test_run